In [2]:
import time
import pandas as pd
import requests
import json
import concurrent.futures
from decouple import config

# Disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
# Bring in earnings and pricing data
earnings_pricing = pd.read_csv("earnings_pricing.csv")
earnings_pricing.head()

,date,symbol,eps,epsEstimated,time,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,changeOverTime
0,11/22/21,A,1.21,1.17,amc,165.000000,165.679993,162.779999,162.779999,162.565887,2110400,2110400,-2.22000,-1.345,163.74666,-0.01345
1,8/17/21,A,1.10,0.99,amc,161.729996,161.839996,159.289993,160.910004,160.498489,2614200,2614200,-0.81999,-0.507,160.68000,-0.00507
2,5/25/21,A,0.71,0.83,amc,133.410004,134.800003,133.009995,133.229996,132.714828,1890600,1890600,-0.18001,-0.135,133.68000,-0.00135
3,2/16/21,A,0.94,0.89,amc,129.309998,131.080002,127.849998,127.949997,127.261604,2132500,2132500,-1.36000,-1.052,128.96000,-0.01052
4,11/23/20,A,0.72,0.93,amc,111.400002,112.419998,109.559998,112.209999,111.423569,3258700,3258700,0.81000,0.727,111.39667,0.00727


In [5]:
earnings_pricing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75955 entries, 0 to 75954
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              75955 non-null  object 
 1   symbol            75955 non-null  object 
 2   eps               75955 non-null  float64
 3   epsEstimated      75955 non-null  float64
 4   time              75955 non-null  object 
 5   open              75955 non-null  float64
 6   high              75955 non-null  float64
 7   low               75955 non-null  float64
 8   close             75955 non-null  float64
 9   adjClose          75955 non-null  float64
 10  volume            75955 non-null  int64  
 11  unadjustedVolume  75955 non-null  int64  
 12  change            75955 non-null  float64
 13  changePercent     75955 non-null  float64
 14  vwap              75955 non-null  float64
 15  changeOverTime    75955 non-null  float64
dtypes: float64(11), int64(2), object(3)
memo

In [7]:
stocks = earnings_pricing.symbol.unique()
print(stocks)

['A' 'AA' 'AAL' ... 'ZUMZ' 'ZUO' 'ZWS']


In [8]:
def get_jsonparsed_data(url):
    """
    Sends a GET request to FMP's Earning Surprise API and returns the resulting data in a dictionary
    """
    # sending get request and saving the response as response object
    response = requests.get(url=url)
    data = json.loads(response.text)
    return data

In [9]:
# For each stock build its url
url_list = []

EOD_API_KEY = config("EOD_API_KEY")

# Paramters
last_date = '2022-01-07'
func = 'sma'
period = 5

for idx, val in enumerate(stocks):
    url = "https://eodhistoricaldata.com/api/technical/{}.US?order=d&fmt=json&to={}&function={}&period={}&api_token={}".format(val, last_date, func, period, EOD_API_KEY)
    url_list.append(url)

In [11]:
# Call FMP API for each URL using Concurrent library
# Run takes 211 seconds ... be patient
with concurrent.futures.ThreadPoolExecutor() as executor:
    res = [executor.submit(get_jsonparsed_data, url) for url in url_list]
    concurrent.futures.wait(res)

In [16]:
stock_test = stocks[244]
test_df = pd.DataFrame.from_records(res[244].result())
test_df["stock"] = stock_test
print(test_df)

            date     sma stock
0     2022-01-07  34.878  AVNS
1     2022-01-06  35.148  AVNS
2     2022-01-05  35.138  AVNS
3     2022-01-04  35.010  AVNS
4     2022-01-03  34.690  AVNS
...          ...     ...   ...
1809  2014-10-31  38.090  AVNS
1810  2014-10-30  38.244  AVNS
1811  2014-10-29  38.584  AVNS
1812  2014-10-28  38.884  AVNS
1813  2014-10-27  39.468  AVNS

[1814 rows x 3 columns]


In [26]:
df = pd.DataFrame()
ve_num = 0
ke_num = 0
for idx, val in enumerate(url_list):
    try:
        res_df = pd.DataFrame.from_records(res[idx].result())
        stock = stocks[idx]
        res_df["stock"] = stock
        df = pd.concat([df, res_df])
    # If value error occurs skip the stock
    except ValueError as ve:
        ve_num += 1
        print(ve)
        pass
    except KeyError as ke:
        ke_num += 1
        message = "KeyError at index: {}, url: {}, occurence number: {}".format(idx, val, ke_num)
        with open("eod_error_logs.txt", "a") as text_file:
            text_file.write(message)
            text_file.write('\n')
        print("KeyError at index: {}".format(idx))

In [ ]:
df_copy = df.copy()

In [24]:
df = df.rename(columns={"stock": "symbol"})
col = df.pop("symbol")
df = df.insert(1, col.name, col)
print(df.info())
print(df.isnull().values.any())

AttributeError: 'NoneType' object has no attribute 'info'

In [22]:
print(earnings_pricing.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75955 entries, 0 to 75954
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              75955 non-null  object 
 1   symbol            75955 non-null  object 
 2   eps               75955 non-null  float64
 3   epsEstimated      75955 non-null  float64
 4   time              75955 non-null  object 
 5   open              75955 non-null  float64
 6   high              75955 non-null  float64
 7   low               75955 non-null  float64
 8   close             75955 non-null  float64
 9   adjClose          75955 non-null  float64
 10  volume            75955 non-null  int64  
 11  unadjustedVolume  75955 non-null  int64  
 12  change            75955 non-null  float64
 13  changePercent     75955 non-null  float64
 14  vwap              75955 non-null  float64
 15  changeOverTime    75955 non-null  float64
dtypes: float64(11), int64(2), object(3)
memo

In [ ]:
total_df = earnings_pricing_df.join(df, on=['date', 'symbol'])